# TP1

In [4]:
import numpy as np

## Exercice 2

In [2]:
dataset = np.loadtxt(
    "DataMining/data/pl.csv",  # Remplacez cette valeur par le chemin d'accès de votre fichier CSV.
    dtype={"names": ("name", "year"), "formats": ("U100", "i4")},
    skiprows=1,  # passez la première ligne, puisque c'est l'en-tête
    delimiter=",",  # le séparateur est une virgule puisqu'il s'agit d'un fichier CSV.
    encoding="UTF-8",  # encodage UTF-8
)
print(len(dataset))

dataset = np.loadtxt(
    "DataMining/data/pl.tsv",  # Remplacez cette valeur par le chemin d'accès de votre fichier TSV.
    dtype={"names": ("name", "year"), "formats": ("U100", "i4")},
    skiprows=1,
    delimiter="\t",  # le séparateur est '\t' puisqu'il s'agit d'un fichier TSV.
    encoding="UTF-8",
)
print(len(dataset))

100
100


## Exercice 3

In [5]:
import urllib.request
import pandas as pd
import json

url = "https://query.wikidata.org/sparql?query=SELECT%20%3FlanguageLabel%20(YEAR(%3Finception)%20as%20%3Fyear)%0AWHERE%0A%7B%0A%20%20%23instances%20of%20programming%20language%0A%20%20%3Flanguage%20wdt%3AP31%20wd%3AQ9143%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP571%20%3Finception%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FlanguageLabel.%0A%20%20FILTER(lang(%3FlanguageLabel)%20%3D%20%22en%22)%0A%7D%0AORDER%20BY%20%3Fyear%0ALIMIT%20100&format=json"
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))

array = []

for data in responsedata["results"]["bindings"]:
    array.append([data["year"]["value"], data["languageLabel"]["value"]])

dataframe = pd.DataFrame(array, columns=["year", "languageLabel"])
dataframe = dataframe.astype(dtype={"year": "<i4", "languageLabel": "<U200"})
print(dataframe)

    year                              languageLabel
0   1942                                 Plankalkül
1   1943                        ENIAC coding system
2   1946                           ENIAC Short Code
3   1946  Von Neumann and Goldstine graphing system
4   1948                          CPC Coding scheme
..   ...                                        ...
95  1967                                       BCPL
96  1967                 Space Programming Language
97  1968                                       CICS
98  1968                                      FOCAL
99  1968                                      Refal

[100 rows x 2 columns]


## Exercice 5

In [13]:
from pandas import json_normalize
import pandas as pd
import json

jsondata = json.load(open("DataMining/data/plparadigm.json"))
array = []

for data in jsondata:
    array.append([data["year"], data["languageLabel"], data["paradigmLabel"]])

dataframe = pd.DataFrame(array, columns=["year", "languageLabel", "paradigmLabel"])
dataframe = dataframe.astype(
    dtype={"year": "int64", "languageLabel": "<U200", "paradigmLabel": "<U200"}
)

grouped = dataframe.groupby(["year", "paradigmLabel"]).agg(["count"])
grouped

# grouped = dataframe.groupby(["paradigmLabel", "year"]).agg(["count"])
# grouped

languageLabel
                                                    count
year paradigmLabel                                       
1948 procedural programming                             1
1949 non-structured programming                         1
     procedural programming                             1
1950 procedural programming                             1
1953 procedural programming                             1
...                                                   ...
1970 imperative programming                             1
     procedural programming                             1
     structured programming                             1
1971 functional programming                             1
     knowledge representation and reasoning             1

[73 rows x 1 columns]

In [14]:
from pandas import json_normalize
import pandas as pd
import json

# Charger les données JSON
jsondata = json.load(open("countrydata.json"))
array = []

# Extraire les données nécessaires
for data in jsondata:
    array.append([data["year"], data["countryLabel"], data["population"]])

# Créer un DataFrame
dataframe = pd.DataFrame(array, columns=["year", "countryLabel", "population"])

# Convertir les types de données appropriés
dataframe["year"] = pd.to_datetime(dataframe["year"], format='%Y')
dataframe["population"] = pd.to_numeric(dataframe["population"])

# Trier le DataFrame par ordre alphabétique de pays et année croissante
dataframe.sort_values(by=["countryLabel", "year"], inplace=True)

# Afficher la population des pays dans l'ordre alphabétique de leur nom et par ordre croissant d'année
print("Population des pays dans l'ordre alphabétique de leur nom et par ordre croissant d'année :")
print(dataframe)

# Obtenir la dernière population disponible de chaque pays
latest_population = dataframe.groupby('countryLabel')['population'].last().reset_index()

# Identifier le pays avec la population la plus faible et la plus élevée (la plus récente)
min_population_country = latest_population.loc[latest_population['population'].idxmin()]['countryLabel']
max_population_country = latest_population.loc[latest_population['population'].idxmax()]['countryLabel']

# Afficher le pays avec la population la plus faible et la plus élevée
print(f"Pays avec la population la plus faible : {min_population_country}")
print(f"Pays avec la population la plus élevée : {max_population_country}")

Population des pays dans l'ordre alphabétique de leur nom et par ordre croissant d'année :
          year countryLabel  population
0   1960-01-01  Afghanistan     8774440
1   1961-01-01  Afghanistan     8953544
2   1962-01-01  Afghanistan     9141783
3   1963-01-01  Afghanistan     9339507
4   1964-01-01  Afghanistan     9547131
..         ...          ...         ...
995 2007-01-01      Bolivia     9676456
996 2008-01-01      Bolivia     9834098
997 2009-01-01      Bolivia     9993406
998 2010-01-01      Bolivia    10156601
999 2011-01-01      Bolivia    10324445

[1000 rows x 3 columns]
Pays avec la population la plus faible : Andorra
Pays avec la population la plus élevée : Bangladesh


In [19]:
import pandas as pd

# Charger le fichier JSON
data = pd.read_json("scienceData.json")

# Créer un DataFrame regroupé par sujet et année
df_annee_sujet = data.groupby(["year", "subjectLabel"]).size().reset_index(name="Nombre d'articles")

# Afficher le résultat
print(f"Le nombre d'articles publiés sur différents sujets chaque année : {df_annee_sujet.to_string()}")

# Trouver le sujet avec le nombre d'articles le plus élevé pour chaque année
df_sujet_principal_annee = df_annee_sujet.sort_values("Nombre d'articles", ascending=False).groupby("year").head(1)

# Afficher le résultat
print(f"Principal sujet d'intérêt : {df_sujet_principal_annee.to_string()}")

# Filtrer les données depuis 2010
df_depuis_2010 = df_annee_sujet[df_annee_sujet["year"] >= 2010]

# Compter le nombre d'articles par sujet
df_top_10_sujets = df_depuis_2010.groupby("subjectLabel").size().sort_values(ascending=False).head(10)

# Afficher le résultat
print(f"Les 10 principaux sujets d'intérêt : {df_top_10_sujets.to_string()}")


Le nombre d'articles publiés sur différents sujets chaque année :     year subjectLabel  Nombre d'articles
0   2011        Earth                 65
1   2011     Universe                 34
2   2012        Earth                 34
3   2012     Universe                 41
4   2013        Earth                 38
5   2013     Universe                 34
6   2014        Earth                 69
7   2014     Universe                 65
8   2015        Earth                 64
9   2015     Universe                 55
10  2015        death                  1
11  2016        Earth                 66
12  2016     Universe                 47
13  2017        Earth                 48
14  2017     Universe                 38
15  2018        Earth                 50
16  2018     Universe                 31
17  2019        Earth                 38
18  2019     Universe                 41
19  2020        Earth                 19
20  2020     Universe                 24
21  2021        Earth           

## Exercice 6

In [7]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil
import os
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 10"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open(os.path.basename(url), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )
    
dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe.image.apply(download_image)

dataframe

,ville,pays,image
0,Erevan,Arménie,http://commons.wikimedia.org/wiki/Special:File...
1,Helsinki,Finlande,http://commons.wikimedia.org/wiki/Special:File...
2,Pskov,Russie,http://commons.wikimedia.org/wiki/Special:File...
3,Copenhague,Danemark,http://commons.wikimedia.org/wiki/Special:File...
4,Québec,Canada,http://commons.wikimedia.org/wiki/Special:File...
5,Pune,Inde,http://commons.wikimedia.org/wiki/Special:File...
6,Riazan,Russie,http://commons.wikimedia.org/wiki/Special:File...
7,Erfurt,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
8,Djouba,Soudan du Sud,http://commons.wikimedia.org/wiki/Special:File...
9,Edmonton,Canada,http://commons.wikimedia.org/wiki/Special:File...
